In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification,Trainer
raw_datasets=load_dataset("glue","sst2")
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
  return tokenizer(examples['sentence'],truncation=True,padding="max_length")


tokenized_datasets=raw_datasets.map(preprocess_function,batched=True)

train_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
eval_subset = tokenized_datasets["validation"].shuffle(seed=42).select(range(100))


training_arg=TrainingArguments(
    output_dir="./bert-sentiment",
    learning_rate=0.00002,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10
)




model=model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)


trainer=Trainer(
    model=model,
    args=training_arg,
    train_dataset=train_subset,
    eval_dataset=eval_subset
)


trainer.train()






/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Step,Training Loss
10,0.693425
20,0.675874
30,0.668459
40,0.632103
50,0.547481
60,0.489975
70,0.351754
80,0.341585
90,0.273131
100,0.297615


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=189, training_loss=0.35858777654234064, metrics={'train_runtime': 312.8759, 'train_samples_per_second': 9.588, 'train_steps_per_second': 0.604, 'total_flos': 789333166080000.0, 'train_loss': 0.35858777654234064, 'epoch': 3.0})

In [2]:

save_directory = "./my_fine_tuned_bert"
trainer.save_model(save_directory)


tokenizer.save_pretrained(save_directory)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('./my_fine_tuned_bert/tokenizer_config.json',
 './my_fine_tuned_bert/tokenizer.json')

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


load_directory = "./my_fine_tuned_bert"


loaded_tokenizer = AutoTokenizer.from_pretrained("./my_fine_tuned_bert")


loaded_model = AutoModelForSequenceClassification.from_pretrained("./my_fine_tuned_bert")





print("Model loaded successfully")

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Model loaded successfully


In [4]:
import torch

def predict_sentiment(text):

    inputs = loaded_tokenizer(text, return_tensors="pt")


    with torch.no_grad():
        outputs = loaded_model(**inputs)


    logits = outputs.logits


    predicted_class_id = torch.argmax(logits, dim=1).item()


    labels = ["Negative", "Positive"]
    return labels[predicted_class_id]


print(predict_sentiment("I love to learn about new things"))
print(predict_sentiment("he is not satisfied"))

Positive
Negative
